# Textstatistik: `textacy`

Neben `spacy` wirst du häufig auch `textacy` zur Vorverarbeitung von Texten einsetzen. `textacy` hat sehr viele ganz unterschiedliche Funktionen, die am Anfang etwas zu überblicken sind.

In diesem Teil wirst du daher einige der Möglichkeiten kennenlernen, für die man `textacy` besonders gut einsetzen kann.

Achtung: Zum aktuellen Zeitpunk (März 2021) funktioniert `textacy` nur eingeschränkt gut mit `spacy` in der Version 3.0 zusammen. Gemeinsam mit dem Autor von `textacy` arbeiten wir an einer Lösung. Sollte es uns nicht gelingen, dauerhaft eine Kompatibilität herzustellen, werden wir ein abgeleitetes Textacy-Paket mit in das Github-Archiv legen. Dann solltest du `textacy` nicht separat installieren.

## Normalisierung

Eine Aufgabe, die du im letzten Kapitel schon selbstständig gelöst hast, kann `textacy` auch erledigen: das Wiederzusammenfügen von Trennungen. Betrachte dazu nochmals den Abschnitt aus dem PDF-Dokument:

In [ ]:
text = """Figure 7:  Schematic inheritance relations and properties for the top-level Self-Description  
Schemas.

© BMWi

132  CORE ARCHITECTURE ELEMENTS

2.5 Catalogue

The concept Self-Description is the foundation of the 
federated GAIA-X Catalogues. Catalogues are the 
main building block for the publication and discovery 
of Self-Descriptions of Assets and Participants. To sat-
isfy Consumer needs and to objectively find the best 
fitting offerings in the tangle of registered Assets, an 
open and transparent query algorithm is implemented 
without any GAIA-X internal ranking. Beside search 
functionality, a graph-based navigation interface is 
provided to traverse the complex tangle of offered 
Services, Nodes and linked Self-Descriptions, includ-
ing the attached claims with chain of trust statements. 
Consumers can verify each Self-Description individu-
ally and decide which one to select in a self-sovereign 
manner – GAIA-X does not act as a runtime interme-
diary or broker.
"""

Mithilfe des `preprocessing`-Moduls von `textacy` kannst du die Trennungen direkt zusammenführen:

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import textacy.preprocessing
text_hyphen = textacy.preprocessing.normalize.hyphenated_words(text)
print(text_hyphen)

Das Ergebnis sieht zum Glück genau aus wie mit unserer eigenen Methode.

`textacy` kann noch weitere Normalisierungen durchführen, wie die Entfernung von mehrfachen *Whitespaces*. Nachdem Leerzeilen auch als Whitespaces betrachtet werden, führt das zu einem Text ohne Leerzeilen:

In [ ]:
print(textacy.preprocessing.normalize.whitespace(text_hyphen))

Hier siehst du, dass durch die Vorverarbeitung tatsächlich Informationen verloren gegangen sind. So ist nun nicht mehr ersichtlich, wo wirklich ein neuer Absatz beginnt. Deshalb solltest du mit der Filterung auch vorsichtig umgehen.

## Basisdaten

Für einige weitere Funktionen von `textacy` betrachten wir diesmal nur dern ersten Paragraph aus dem Neujahrs-Artikel:

In [ ]:
p1 = "Doch das Ende des Jahres 2020 birgt auch Hoffnung, dass durch die Vakzinen \
gegen Covid-19 wieder Normalität einkehre – wie immer die auch aussehen mag \
– und wir uns um anderes Dringliches kümmern oder einfach entspannen \
können. Und dass durch den im Januar anstehenden Bewohnerwechsel im \
Weißen Haus zu Washington D.C. das offizielle Herumgetrumpel auf dem \
gesunden Menschenverstand ein Ende finden möge."

## Keywords in context

`textacy` hat eine Funktion `KWIC`, was für *Keywords in Context* steht und kann dir damit anzeigen, in welchem Kontext ein Wort vorkommt:

In [ ]:
list(textacy.extract.kwic.keyword_in_context(p1, "Hoffnung", window_width=35))

Diese Funktion ist auch ganz einfach selbst implementiert (wie du später noch genauer sehen wirst), kann aber sehr nützlich sein.

## n-Gramme und Noun Phrases

`texacy` kann sehr eng mit `spacy` zusammenarbeiten und auch die `spacy`-Dokumente übernehmen. Damit kannst du dir schnell interessante Statistiken erschließen:

In [ ]:
de = textacy.load_spacy_lang("de_core_news_lg")
doc = textacy.make_spacy_doc(p1, lang=de)
doc._.preview

`textacy`macht es dir sehr einfach, Wortkombinationen (sog. *n-Gramme*) zu ermitteln und dabei gleich bestimmte Tokentypen von `spacy` auszufiltern:

In [ ]:
list(textacy.extract.ngrams(doc, 3, filter_stops=True, filter_punct=True, 
                            filter_nums=False))

Auch die aus `spacy` bekannten Entitäten kannst du dir schnell ermitteln lassen:

In [ ]:
list(textacy.extract.entities(doc, drop_determiners=True))

`textacy` kann sog. *Noun Phrases* ermitteln. Im Gegensatz zu früheren Versionen, die dabei auf *Regular Expressions* aufgesetzt hat, wird jetzt der *Dependency Parser* von `spacy` dafür verwendet:

In [ ]:
list(textacy.extract.noun_chunks(doc))

## TextRank und Worte zählen

Der TextRank-Algorithmus versucht, besonders wichtige Konstrukte zu bewerten und diese entsprechend zu sortieren:

In [ ]:
import textacy.extract.keyterms
textacy.extract.keyterms.textrank(doc, normalize="lemma", topn=10)

Mihilfe der Methode `to_bag_of_term` kannst du die Häufigkeit der Wörter zählen. Der `Counter` von Python bietet dir dann komfortable Zugriffsmöglichkeiten:

In [ ]:
from collections import Counter
bot = Counter(doc._.to_bag_of_terms(ngs=(1, 2, 3), ents=True, 
                                    weighting="count")) #, as_strings=True))
bot.most_common(10)

## Mehrere Dokumente verarbeiten

Was mit einem Dokument funktioniert, kannst du natürlich auch für mehrere Dokumente des gesamten Corpus durchführen. Dazu wählst du zunächst 100 Dokumente aus der Datenbank aus:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
# 100 Dokumente selektieren
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM articles \
                  ORDER BY datePublished DESC LIMIT 100", sql)
df["full_text"] = df["title"] + "\n" + df["header"] + "\n" + df["text"]

`textacy` hat eine eigene Datenstruktur für einen `Corpus`, die du mit den Dokumenten initalisieren kannst:

In [ ]:
import textacy
corpus = textacy.Corpus("de_core_news_lg", 
                        data = list(df["full_text"].values))

Damit stehen dir gleich Statistik-Informationen zur Verfügung:

In [ ]:
corpus.n_docs, corpus.n_sents, corpus.n_tokens

Auch die Worthäufigkeiten selbst kannst du von `textacy` zählen lassen und packst sie danach am besten wieder in einen `Counter`: 

In [ ]:
word_counts = Counter(corpus.word_counts(by="lemma_"))
word_counts.most_common(10)

## Noch viel mehr Möglichkeiten

`textacy` kann noch deutlich mehr. Allerdings werden fast alle weitergehenden Funktionen von `textacy` auch in anderen Notebooks vorgestellt. Dennoch kannst du `textacy` gut als *Schweizer Taschenmesser* nutzen und viele Funktionen darin finden..

## Datensets zum Ausprobieren

In [ ]:
import textacy.datasets
ds = textacy.datasets.CapitolWords()

In [ ]:
ds.download()
records = ds.records(speaker_name={"Hillary Clinton", "Barack Obama"})

In [ ]:
list(records)